<h1> Empirical estimation of IPMs (integral probability metrics): prerequisite<span class="tocSkip"></span></h1>

Author: [Sylvain Combettes](https://github.com/sylvaincom).

Last update: Feb 3, 2020.

GitHub repository: [here](https://github.com/sylvaincom/comparison-distributions).

---
This notebook is the prerequisite of my notebook `ipm.ipynb` about the empirical estimation of IPMs (integral probability metrics) and completes my report on the _Comparison of Empirical Probability Distributions_. One can consider this notebook as an introduction to `ipm.ipynb`.

As $f$-divergences input probability distributions, IPMs input samples drawn from the probability distributions. Actually, for IPMs, we only focus on the Kantorovich metric.

<br/>

<div class="alert alert-info"><h4>README<span class="tocSkip"></span></h4>

<p> We recommend reading the report before reading this notebook, as most explanations are not duplicated here.</p>

<p>
The best way to open this Jupyter Notebook is to use the table of contents from the extensions called <code>nbextensions</code>. See <a href="https://towardsdatascience.com/4-awesome-tips-for-enhancing-jupyter-notebooks-4d8905f926c5">4 Awesome Tips for Enhancing Jupyter Notebooks</a> by George Seif.
    
The Python version is 3.7.3.
</p></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminary:-Solving-a-linear-programming-problem-using-PuLP" data-toc-modified-id="Preliminary:-Solving-a-linear-programming-problem-using-PuLP-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminary: Solving a linear programming problem using <code>PuLP</code></a></span></li><li><span><a href="#Example-of-empirical-estimation-of-the-Kantorovich-metric-$W$" data-toc-modified-id="Example-of-empirical-estimation-of-the-Kantorovich-metric-$W$-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example of empirical estimation of the Kantorovich metric $W$</a></span><ul class="toc-item"><li><span><a href="#Getting-the-canonical-form-of-the-linear-programming-problem" data-toc-modified-id="Getting-the-canonical-form-of-the-linear-programming-problem-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Getting the canonical form of the linear programming problem</a></span></li><li><span><a href="#Using-PuLP-to-solve-the-obtained-linear-programming-problem" data-toc-modified-id="Using-PuLP-to-solve-the-obtained-linear-programming-problem-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Using <code>PuLP</code> to solve the obtained linear programming problem</a></span></li></ul></li></ul></div>

<h2> Imports<span class="tocSkip"></span></h2>

In [1]:
# pip install pulp
from pulp import *
import numpy as np

# Preliminary: Solving a linear programming problem using `PuLP`

As I am new to `PuLP`, this section constitutes a short introduction of how `PuLP` works.
This section is inspired from the official `PuLP` [documentation](https://pypi.org/project/PuLP/).

Use `LpVariable()` to create new variables. To create a variable $0 \leq x \leq 3$:

In [2]:
x = LpVariable("x", 0, 3)

To create a variable $0 \leq y \leq 1$:

In [3]:
y = LpVariable("y", 0, 1)

Use `LpProblem()` to create new problems. Create `“myProblem”`:

In [4]:
prob = LpProblem("myProblem", LpMinimize)

Combine variables to create expressions and constraints, then add them to the problem:

In [5]:
prob += x + y <= 2

If you add an expression (not a constraint), it will become the objective:

In [6]:
prob += -4*x + y

Our `prob` variable contains all the data of our linear programming problem:

In [7]:
print(prob)

myProblem:
MINIMIZE
-4*x + 1*y + 0
SUBJECT TO
_C1: x + y <= 2

VARIABLES
x <= 3 Continuous
y <= 1 Continuous



To solve with the default included solver:

In [8]:
status = prob.solve()

Display the status of the solution:

In [9]:
LpStatus[status]

'Optimal'

You can get the value of the variables using `value()`:

In [10]:
print('The solution is (%1.2f, %1.2f) and the objective function is %1.2f'
      % (value(x), value(y), value(prob.objective)))

The solution is (2.00, 0.00) and the objective function is -8.00


We can check this result graphically using [GeoGebra](https://www.geogebra.org/) and this [tutorial](https://www.youtube.com/watch?v=4vvlmSJqJto). With GeoGebra, we obtained the same result as with `PuLP`:

<img src="https://raw.githubusercontent.com/sylvaincom/comparison-distributions/master/img/geogebra.png" width="600" height="539.6">

For more information on `PuLP`, see examples on [GitHub](https://github.com/coin-or/pulp/tree/master/examples).

# Example of empirical estimation of the Kantorovich metric $W$

We take the example from the report at subsection II.3.2. Our linear programming problem in the canonical form is:

\begin{equation}
\begin{cases}
    \max \left(c^Ta\right) \\
    Ma \leq b
\end{cases}
\end{equation}

With $N=4$, we have:

\begin{equation}
c =
\begin{pmatrix}
    \widetilde{Y}_1 \\
    \widetilde{Y}_2 \\
    \widetilde{Y}_3 \\
    \widetilde{Y}_4
\end{pmatrix} \quad \quad
M = \begin{pmatrix}
        1 & -1 & 0 & 0 \\
        -1 & 1 & 0 & 0 \\
        1 & 0 & -1 & 0 \\
        -1 & 0 & 1 & 0 \\
        1 & 0 & 0 & -1 \\
        -1 & 0 & 0 & 1 \\
        0 & 1 & -1 & 0 \\
        0 & -1 & 1 & 0 \\
        0 & 1 & 0 & -1 \\
        0 & -1 & 0 & 1 \\
        0 & 0 & 1 & -1 \\
        0 & 0 & -1 & 1
    \end{pmatrix} \quad \quad
b = \begin{pmatrix}
        \rho\left(X_1, X_2\right) \\
        \rho\left(X_1, X_2\right) \\
        \rho\left(X_1, X_3\right) \\
        \rho\left(X_1, X_3\right) \\
        \rho\left(X_1, X_4\right) \\
        \rho\left(X_1, X_4\right) \\
        \rho\left(X_2, X_3\right) \\
        \rho\left(X_2, X_3\right) \\
        \rho\left(X_2, X_4\right) \\
        \rho\left(X_2, X_4\right) \\
        \rho\left(X_3, X_4\right) \\
        \rho\left(X_3, X_4\right)
    \end{pmatrix}
\end{equation}

We take the same uniform distributions as in the report at subsection II.1.3.

We choose the samples $\left\{X_1^{(1)}, X_2^{(1)}, \ldots, X_m^{(1)}\right\}$ drawn randomly from $\mathbb{P}$, noted `X_p`, to be:

In [11]:
m = 2
a = 1
b = 2
np.random.seed(1) # random seed for reproducability
X_p = np.random.uniform(a, b, m)
print('X_p is: \n', X_p)

X_p is: 
 [1.417022   1.72032449]


We choose the samples $\left\{X_1^{(2)}, X_2^{(2)}, \ldots, X_n^{(2)}\right\}$ drawn randomly from $\mathbb{Q}$, noted `X_q`, to be:

In [12]:
n = 2
r = 9
s = 10
np.random.seed(1) # random seed for reproducability
X_q = np.random.uniform(r, s, n)
print('X_q is: \n', X_q)

X_q is: 
 [9.417022   9.72032449]


Thus, we have:

In [13]:
X = np.concatenate((X_p, X_q), axis=0)
print('X is: \n', X, '\n')

N = m+n
print('N is: \n', N)

X is: 
 [1.417022   1.72032449 9.417022   9.72032449] 

N is: 
 4


## Getting the canonical form of the linear programming problem

In [14]:
def construct_c(X_p, X_q):
    m = len(X_p)
    n = len(X_q)
    
    Y = [1/m]*m + [-1/n]*n
    
    return np.asarray(Y)

In [15]:
c = construct_c(X_p, X_q)
print('c is: \n', c)

c is: 
 [ 0.5  0.5 -0.5 -0.5]


In [16]:
def rho(x,y):
    return abs(x-y)

In [17]:
def construct_b(X_p, X_q):
    
    X = np.concatenate((X_p, X_q), axis=0)
    N = len(X)
    
    b_part = []
    for i in range(N):
        for j in range(i+1, N):
            b_part.append(rho(X[i], X[j]))
    
    # Now, we duplicate each row to obtain a list of size 2*N
    b = []
    for i in range(N):
        b.append(b_part[i])
        b.append(b_part[i])
    
    return b

In [18]:
b = construct_b(X_p, X_q)
print('b is: \n', b)

b is: 
 [0.3033024887395839, 0.3033024887395839, 8.0, 8.0, 8.303302488739584, 8.303302488739584, 7.696697511260416, 7.696697511260416]


In [19]:
def construct_M(X_p, X_q):
    
    X = np.concatenate((X_p, X_q), axis=0)
    N = len(X)
    
    M = []
    for i in range(N):
        for j in range(i+1, N):
            l_M_1 = [0]*N
            l_M_1[i] = 1
            l_M_1[j] = -1
            M.append(l_M_1)
            l_M_2 = [0]*N
            l_M_2[i] = -1
            l_M_2[j] = 1
            M.append(l_M_2)
    M = np.asarray(M)

    return M.astype(int)

In [20]:
M = construct_M(X_p, X_q)
print('M is: \n', M)

M is: 
 [[ 1 -1  0  0]
 [-1  1  0  0]
 [ 1  0 -1  0]
 [-1  0  1  0]
 [ 1  0  0 -1]
 [-1  0  0  1]
 [ 0  1 -1  0]
 [ 0 -1  1  0]
 [ 0  1  0 -1]
 [ 0 -1  0  1]
 [ 0  0  1 -1]
 [ 0  0 -1  1]]


## Using `PuLP` to solve the obtained linear programming problem

We rely on [test5](https://github.com/coin-or/pulp/blob/master/examples/test5.py) of the official `PuLP` examples to define our LP (linear programming) problem with matrices.

From the previous subsection, we have:

In [21]:
print('N is: \n', N, '\n')
print('c is: \n', c, '\n')
print('M is: \n', M, '\n')
print('b is: \n', b)

N is: 
 4 

c is: 
 [ 0.5  0.5 -0.5 -0.5] 

M is: 
 [[ 1 -1  0  0]
 [-1  1  0  0]
 [ 1  0 -1  0]
 [-1  0  1  0]
 [ 1  0  0 -1]
 [-1  0  0  1]
 [ 0  1 -1  0]
 [ 0 -1  1  0]
 [ 0  1  0 -1]
 [ 0 -1  0  1]
 [ 0  0  1 -1]
 [ 0  0 -1  1]] 

b is: 
 [0.3033024887395839, 0.3033024887395839, 8.0, 8.0, 8.303302488739584, 8.303302488739584, 7.696697511260416, 7.696697511260416]


We define our LP problem:

In [22]:
prob = LpProblem("LP problem for estimating the Kantorovich metric", LpMaximize)

We create our vector of $N$ continuous variables:

In [23]:
a = LpVariable.matrix("a", list(range(N)))

We define our objective function:

In [24]:
prob += lpDot(c, a)

We define our number of constraints:

In [25]:
p = 2*N

We define our constraints:

In [26]:
for i in range(p):
    prob += lpDot(M[i], a) <= b[i]

We print our LP problem:

In [27]:
print(prob)

LP problem for estimating the Kantorovich metric:
MAXIMIZE
0.5*a_0 + 0.5*a_1 + -0.5*a_2 + -0.5*a_3 + 0.0
SUBJECT TO
_C1: a_0 - a_1 <= 0.30330248874

_C2: - a_0 + a_1 <= 0.30330248874

_C3: a_0 - a_2 <= 8

_C4: - a_0 + a_2 <= 8

_C5: a_0 - a_3 <= 8.30330248874

_C6: - a_0 + a_3 <= 8.30330248874

_C7: a_1 - a_2 <= 7.69669751126

_C8: - a_1 + a_2 <= 7.69669751126

VARIABLES
a_0 free Continuous
a_1 free Continuous
a_2 free Continuous
a_3 free Continuous



We solve our problem:

In [28]:
prob.solve()

1

We print the status of our solved LP problem:

In [29]:
print("Status:", LpStatus[prob.status])

Status: Optimal


We print the value of the variables at the optimum:

In [30]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

a_0 = 0.0
a_1 = 0.0
a_2 = -7.6966975
a_3 = -8.3033025


We print the value of the objective:

In [31]:
print("objective=", value(prob.objective))

objective= 8.0


After solving, the value of objective function is the estimation of our Kantorovich metric.

As in formula (II.11) of the report, the value is close close to $8$ (because $d=1$).